In [1]:
import pandas as pd
import sys
sys.path.append("../../helper_functions")
import opstack_metadata_utils as ops
import duneapi_utils as d
import google_bq_utils as bqu
import clickhouse_utils as ch
sys.path.pop()

import dotenv
import os
dotenv.load_dotenv()

True

In [2]:
# Read the CSV file
df = pd.read_csv('chain_metadata_raw.csv')

table_name = 'op_stack_chain_metadata'

In [3]:
import math

def convert_to_int_or_keep_string(value):
    try:
        float_value = float(value)
        if math.isnan(float_value):  # Check if value is NaN
            return None
        if float_value.is_integer():
            val = str(int(float_value))  # Convert to int and then to string
        else:
            val = str(value)  # Convert to string
    except ValueError:
        val = str(value)  # Convert to string
    if val.endswith('.0'):
        val = val[:-2]  # Remove the last two characters
    return val


In [4]:
# Trim columns
df.columns = df.columns.str.replace(" ", "").str.strip()
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# Datetime
df['public_mainnet_launch_date'] = pd.to_datetime(df['public_mainnet_launch_date'], errors='coerce')
df['op_chain_start'] = pd.to_datetime(df['op_chain_start'], errors='coerce')
# ChainID
# Apply the function to the column
df['mainnet_chain_id'] = df['mainnet_chain_id'].apply(convert_to_int_or_keep_string)
# df['mainnet_chain_id'] = int(df['mainnet_chain_id'])
#Generate Alignment Column
df = ops.generate_alignment_column(df)

# Replace NaN values in object type columns with an empty string
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns] = df[object_columns].fillna('')

# Save the cleaned DataFrame to a new CSV file
df.to_csv('../outputs/chain_metadata.csv', index=False)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_10643/78957915.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_10643/78957915.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['public_mainnet_launch_date'] = pd.to_datetime(df['public_mainnet_launch_date'], errors='coerce')
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_10643/78957915.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['op_chain_start'] = pd.to_datetime(df['op_chain_start'], errors='coerce')


In [5]:
# df.dtypes

In [6]:
#check chain id
df[df['display_name'] =='OP Mainnet']

,chain_name,display_name,mainnet_chain_id,public_mainnet_launch_date,op_based_version,chain_type,op_chain_start,has_mods,raas_deployer,rpc_url,...,oso_schema,batchinbox_from,batchinbox_to,outputoracle_from,outputoracle_to_proxy,l1_standard_bridge,optimism_portal,dispute_game_factory,is_op_chain,alignment
0,op,OP Mainnet,10,2021-11-11,modern,OP Mainnet,2021-06-23,no,OP Mainnet,https://mainnet.optimism.io,...,optimism,0x6887246668a3b87F54DeB3b94Ba47a6f63F32985,0xff00000000000000000000000000000000000010,0x473300df21D047806A082244b417f96b32f13A33,0xdfe97868233d1aa22e815a266982f2cf17685a27,0x99C9fc46f92E8a1c0deC1b1747d010903E884bE1,,0xe5965Ab5962eDc7477C8520243A95517CD252fA9,True,OP Chain


In [7]:
# Post to Dune API
d.write_dune_api_from_pandas(df, table_name + '_info_tracking',\
                             'Basic Info & Metadata about OP Stack Chains, including forks')
#BQ Upload
bqu.write_df_to_bq_table(df, table_name)

#CH Upload
ch.write_df_to_clickhouse(df, table_name, if_exists='replace')

uploading op_stack_chain_metadata_info_tracking
Response status code: 200
Response content: b'{"success":true,"table_name":"op_stack_chain_metadata_info_tracking"}'
table at: dune.oplabspbc.dataset_op_stack_chain_metadata_info_tracking
Start Writing api_table_uploads.op_stack_chain_metadata
Data loaded successfully to api_table_uploads.op_stack_chain_metadata
Existing data in table 'op_stack_chain_metadata' has been deleted.
Data written to table 'op_stack_chain_metadata' successfully, replacing previous data.
